In [2]:
import json
from typing import Any

from models import GameData

with open("./daten_severin_20250901.json", "r") as f:
    raw_data: dict[str, Any] = json.load(f)

data = [GameData(**item) for item in raw_data]  # type: ignore

In [3]:
import re

from models import Bot


def get_human_vote(message: str, bots: list[Bot]) -> str | None:
    bot_colors = {bot.color for bot in bots}
    # Find all colors mentioned in the message
    mentioned_colors = set(re.findall(r"Player (\w+)", message))
    # Human color is the one not in bot_colors
    human_colors = mentioned_colors - bot_colors
    if not human_colors:
        return None
    human_color = human_colors.pop()
    # Find which bot the human accused
    match = re.search(rf"Player {human_color} accused (\w+)", message)
    if match:
        voted_color = match.group(1)
        # Find bot name by color
        for bot in bots:
            if bot.color == voted_color:
                return bot.name
    return None

In [4]:
from collections import Counter

votes: list[str | None] = []
for d in data:
    vote = get_human_vote(d.messages[-1].message, d.bots)
    if vote is None:
        pass
        # print(d.messages[-1].message)
    votes.append(vote)

vote_counts = Counter(votes)
print(vote_counts)

Counter({'AllTalker': 38, 'fourminds': 29, None: 11})


In [5]:
from statistics import mean

# Collect message counts per game
message_counts = [len(d.messages) for d in data]

# Group message counts by vote
vote_to_msg_counts: dict[str, list[int]] = {}
for v, msg_count in zip(votes, message_counts):
    if v is not None:
        vote_to_msg_counts.setdefault(v, []).append(msg_count)

# Calculate averages
for vote_name, counts in vote_to_msg_counts.items():
    avg = mean(counts)
    print(f"{vote_name}: {len(counts)} games, average messages: {avg:.2f}")

fourminds: 29 games, average messages: 35.66
AllTalker: 38 games, average messages: 40.18


In [6]:
from datetime import datetime

# Calculate time deltas (in seconds) for each game
time_deltas: list[float] = []
for d in data:
    start = datetime.fromisoformat(d.starttime)
    end = datetime.fromisoformat(d.updateTS)
    delta = (end - start).total_seconds()
    time_deltas.append(delta)

# Group time deltas by vote
vote_to_time_deltas: dict[str, list[float]] = {}
for v, delta in zip(votes, time_deltas):
    if v is not None:
        vote_to_time_deltas.setdefault(v, []).append(delta)

# Calculate averages
for vote_name, deltas in vote_to_time_deltas.items():
    avg_time = mean(deltas)
    print(f"{vote_name}: {len(deltas)} games, average duration: {avg_time / 60:.2f} minutes")

fourminds: 29 games, average duration: 5.21 minutes
AllTalker: 38 games, average duration: 4.82 minutes
